# Speed and Alignment Notebook Guide
This notebook focuses on calculating the speed of the right wrist and aligning it with pitch data.

## Load Data
In this section, the data is loaded into the notebook for further processing. You can adjust the source file paths as needed.

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

In [ ]:
# Load wrist keypoint data
keypoints_df = pd.read_pickle('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/MOTION_TRACKING_FILES/5012_I_keypoints.pkl')

# Load pitch data
pitch_df = pd.read_pickle('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SOUND_FILES/5012_I_pitch.pkl')

## Calculate Unsmoothed Speed of Right Wrist
Here, the speed of the right wrist is calculated without any smoothing. Parameters such as the sampling rate can be adjusted.

In [ ]:
# Convert keypoints data to DataFrame
keypoints_df = pd.DataFrame(keypoints_df)

# Convert pitch data to DataFrame
pitch_df = pd.DataFrame({'time_ms': pitch_df['time_ms'], 'f0': pitch_df['f0']})

# Calculate the unsmoothed speed of the right wrist
keypoints_df['delta_time'] = keypoints_df['time_ms'].diff()
keypoints_df['delta_x'] = keypoints_df['right_wrist_x'].diff()
keypoints_df['delta_y'] = keypoints_df['right_wrist_y'].diff()

# Speed = sqrt(delta_x^2 + delta_y^2) / delta_time
keypoints_df['speed_unsmooth'] = np.sqrt(keypoints_df['delta_x']**2 + keypoints_df['delta_y']**2) / (keypoints_df['delta_time'] / 1000)  # in unit/s

# Show pitch and keypoint data
pitch_df
keypoints_df

## Apply Savitzky-Golay Filtering
The Savitzky-Golay filtering method is applied to smooth the speed data. You can adjust the window size and polynomial order.

In [ ]:
keypoints_df['speed_smooth'] = [0 if x < np.percentile(keypoints_df['speed_unsmooth'], 25) else x for x in keypoints_df['speed_unsmooth']]
keypoints_df['speed_smooth'] = savgol_filter(keypoints_df['speed_unsmooth'], 11, 1)
keypoints_df

## Align Speed Data with Pitch Data
This section aligns the calculated speed data with the pitch data. Any time offsets or scaling factors can be adjusted here.

In [ ]:
# Upsample the speed data to match the 10 ms sample rate of the pitch data
keypoints_df_upsampled = keypoints_df.set_index('time_ms').reindex(range(0, int(keypoints_df['time_ms'].iloc[-1]) + 1, 10)).interpolate().reset_index()

# Merge the upsampled speed data with the pitch data
aligned_df = pd.merge(keypoints_df_upsampled, pitch_df, on='time_ms', how='inner')

# Drop unnecessary columns
aligned_df = aligned_df.drop(columns=['delta_time', 'delta_x', 'delta_y'])

## Save Aligned Data as CSV
Finally, the aligned data is saved as a CSV file. You can specify the destination path for the output file.

In [ ]:
# Save the aligned data as a CSV file
aligned_df.to_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5012_I_processed_data.csv', index=False)
aligned_df.head()

## Plot Data


In [ ]:
# Create a plot
fig, ax1 = plt.subplots(figsize=(15, 6))

# Plot unsmoothed and smoothed speed
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Speed', color='tab:blue')
ax1.plot(aligned_df['time_ms'], aligned_df['speed_unsmooth'], label='Unsmoothed Speed', color='tab:blue')
ax1.plot(aligned_df['time_ms'], aligned_df['speed_smooth'], label='Smoothed Speed', color='tab:cyan')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# # # Create another y-axis for the pitch data
# ax2 = ax1.twinx()
# ax2.set_ylabel('Pitch (f0)', color='tab:red')
# ax2.plot(aligned_df['time_ms'], aligned_df['f0'], label='Pitch', color='tab:red')
# ax2.tick_params(axis='y', labelcolor='tab:red')

# Add title and legend
plt.title('Unsmoothed Speed, Smoothed Speed, and Pitch over Time')
ax1.legend(loc='upper left')
# ax2.legend(loc='upper right')

plt.show()